In [1]:
import pandas as pd
import numpy as np
import operator
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
import quadratic_weighted_kappa
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import feature_generator
import xgboost as xgb
from scipy import optimize
import os.path
from NN import NN
from XgBoost import XGBoostModel
from sklearn.linear_model import Ridge
from CutPoints import CutPointOptimizer

Using Theano backend.


/Library/Python/2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')  

# train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 10000, True)
# featureImpDf = pd.read_csv('FeatureImportance.csv')
# importantFeatures = featureImpDf[featureImpDf['Importance'] > 11]
# importantFeatures = [column for column in train.columns if column in importantFeatures['Feature'].values]
# train = train[importantFeatures]
# test = test[importantFeatures]

train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)

dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')   

Generating dummies...
Filling in missing values...
Scaling...


In [ ]:
# model = XGBoostModel(700, 7, 0.025, 0.65, "reg:linear", 25)
# model.fit(train, labels)
print train.shape

In [ ]:
importance = model.bst.get_fscore()

In [ ]:
importance = sorted(importance.items(), key=operator.itemgetter(1))
for item in importance:
    print item
# print train.shape

In [ ]:
def WritePredictionsToFile(model, modelName):
    
    kf = KFold(len(train), 3)
    num = 1
    for train_index, test_index in kf:
        foldFile = 'fold%s.csv' % str(num)
        if os.path.isfile(foldFile):
            predictionsDF = pd.read_csv(foldFile)  
        else:
            predictionsDF = pd.DataFrame()
          
        xTrain = train.iloc[train_index].values
        yTrain = labels.iloc[train_index]      
        model.fit(xTrain, yTrain)
        trainPredictions = model.predict(xTrain)
        
        xValidate = train.iloc[test_index].values
        yValidate = labels.iloc[test_index]
        predictions = model.predict(xValidate)
        predictionsDF[modelName] = predictions
        
        print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
        print quadratic_weighted_kappa.quadratic_weighted_kappa(trainPredictions, yTrain)
        
#         youngTrainPredictions = [trainPredictions[i] for i in range(len(xTrain)) if i in youngInd]
#         oldTrainPredictions = [trainPredictions[i] for i in range(len(trainPredictions)) if i in oldInd]
#         youngYTrain = yTrain.iloc[youngInd] 
                        
        predictionsDF.to_csv(path_or_buf=foldFile, index=False)
   
        predictionsFile = 'testPredictions%s.csv' % str(num)
        if os.path.isfile(predictionsFile):
            testDF = pd.read_csv(predictionsFile)   
        else:
            testDF = pd.DataFrame()
            
        xTest = test.values
        testPredictions = model.predict(xTest)
        testDF[modelName] = testPredictions
        testDF.to_csv(path_or_buf=predictionsFile, index=False)
    
        num += 1

In [ ]:
WritePredictionsToFile(LogisticRegression(), 'LogisticRegression')
# WritePredictionsToFile(XGBoostModel(700, 10, 0.025, 0.65, "reg:linear"), 'XGBoostRegLin')

# WritePredictionsToFile(XGBoostModel(0.3, 1, 0, 700), 'XGBoostLinear')
# WritePredictionsToFile(RandomForestRegressor(n_estimators=10, max_depth=10), 'RandomForest')
# WritePredictionsToFile(BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=20), 'BaggingDescisionTrees_n_estimators=20')
# WritePredictionsToFile(BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0), 'BaggingDescisionTreeClassifiers_n_estimators=20')
# WritePredictionsToFile(BaggingRegressor(base_estimator=LinearRegression(), n_estimators=10), 'BaggingLinearRegression_n_estimators=10')

In [ ]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    
# train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 100)
train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)
WritePredictionsToFile(NN(inputShape = train.shape[1], layers = [128, 64], dropout = [0.5, 0.5], activation='sigmoid', loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 8), 'Keras')

In [ ]:

scorer = make_scorer(quadratic_weighted_kappa.quadratic_weighted_kappa)
# print len(features)
# print len(dummyVariables)

In [ ]:
def qwk_scorer(estimator, X, Y):
    predictions = np.clip(estimator.predict(X), 1, 8)
    return quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.20, random_state=0)
bcf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0)
bcf.fit(X_train, y_train)
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [15, 20, 25], 'max_samples': [0.9, 1.0], 'max_features': [0.9, 1.0]}]
# clf = GridSearchCV(SVR(kernel='rbf', max_iter=1000, epsilon=0.49, tol=0.01, verbose=True), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf = GridSearchCV(BaggingClassifier(base_estimator=DecisionTreeClassifier()), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf.fit(X_train, y_train)
# SVR(kernel='rbf', max_iter=1, , tol=0.01, verbose=True)  
#C=5, g=0.1
# TestQWK: 0.24807920235173586
#C=2, g=0.05, qwk = 0.21
#C=5, g=0.05, qwk = 0.25465662455597715
# do C 

In [ ]:
print clf.best_estimator_

In [ ]:
bcp = bcf.predict(X_test)
print quadratic_weighted_kappa.quadratic_weighted_kappa(bcp, y_test)

In [ ]:
print clf.best_params_
clfPredictions = np.clip(clf.predict(X_test), 1, 8)

# dataPoints = list()

# folds = (2, 5, 10, 20)
# for K in folds:
#     _, testQwk, trainQwk = GetBestModel(GenerateNewCombinedModel, features, K)
#     dataPoints.append((K, testQwk, trainQwk))

In [ ]:
print np.min(clfPredictions)
print np.max(clfPredictions)
print clfPredictions

In [38]:
class CombinedModel:
    
    def __init__(self, modelsFromFile, modelsToCalculate):
        self.modelsToCalculate = modelsToCalculate 
        self.modelsFromFile = modelsFromFile
        self.stackingModel = LinearRegression()
        self.boost = [0.652626727963, 0.652281118598, 0.651128156224, 0.651189716136, 0.662062433355]
        self.keras = [0.621456525096, 0.640965818589, 0.64586691993, 0.66517675706, 0.679622912852]
        
    def fit(self, X, Y, fileName, num):     
        stackingTrainData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))    
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            model.fit(X, Y, num)
            predictions = model.predict(X)
            stackingTrainData[:,i] = predictions
            df[model] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingTrainData[:,colsToChange] = df[self.modelsFromFile].values
            
#         self.stackingModel.fit(stackingTrainData, Y)
#         predictions = self.stackingModel.predict(stackingTrainData)
#         predictions = np.add(stackingTrainData[:,0], stackingTrainData[:,1]) / 2
        predictions = stackingTrainData[:,0]
#         predictions = np.add(self.boost[num] * stackingTrainData[:,0], self.keras[num]*stackingTrainData[:,1]) / (self.boost[num] + self.keras[num])
        
        initialCutPoints = np.array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])
        cpo = CutPointOptimizer(predictions, Y)
        self.cutPoints = optimize.fmin(cpo.qwk, initialCutPoints)

        predictions = np.searchsorted(self.cutPoints, predictions) + 1   

        trainQwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)
        print "Train QWK: %s\n" % trainQwk
                           
    def predict(self, X, fileName, num):
        stackingData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            predictions = model.predict(X)
            stackingData[:,i] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingData[:,colsToChange] = df[self.modelsFromFile].values
            
#         predictions = self.stackingModel.predict(stackingData)
#         predictions = np.add(stackingData[:,0], stackingData[:,1]) / 2
#         predictions = np.add(self.boost[num] * stackingData[:,0], self.keras[num]*stackingData[:,1]) / (self.boost[num] + self.keras[num])
        predictions = stackingData[:,0]
        print "CUT POINTS"
        print self.cutPoints
        return np.searchsorted(self.cutPoints, predictions) + 1      

In [ ]:
folds = 5
kf = KFold(len(train), folds)
num = 1
combinedModels = list()
qwks = list()

cpa = np.ndarray((5, 7))
# cpa[0,:] = [1.94094589,  2.77827061, 4.47922564,  5.59375133,  5.98938611,  6.86875996, 7.45790894]
cpa[0,:] = [2.27420049,  3.32644001,  3.22110338,  5.37172015,  6.01216561,  6.68318259, 7.4242825]
cpa[1,:] = [1.93043217,  3.06875824,  4.8242938,   5.40717815,  6.17241788,  6.97729218, 7.46620894]
cpa[2,:] = [1.23315182,  3.39688586,  4.6380404,   5.37977014,  6.16224374,  6.73081194, 7.48322552]
cpa[3,:] = [2.33061803,  3.5055729,   4.52173829,  5.24184554,  6.1711789,   6.96746293, 7.7211743]
cpa[4,:] = [2.26716625,  3.59018596,  4.3780072,   5.21537676,  5.95488354,  6.72023874, 7.5133742]

nnModel = NN(inputShape = train.shape[1], cutPointArray=cpa, layers = [250, 75, 25], dropout = [0.5, 0.5, 0.25], activation='relu', patience=5, loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 50)
 
for train_index, test_index in kf:
    
    combinedModel = CombinedModel([], [nnModel])
#     combinedModel = CombinedModel(['Keras'], [])
    
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)

    xTrain = train.iloc[train_index].values
    yTrain = labels.iloc[train_index]      
    xValidate = train.iloc[test_index].values
    yValidate = labels.iloc[test_index]

    combinedModel.fit(xTrain, yTrain, trainFile, num-1)
    predictions = combinedModel.predict(xValidate, validateFile, num-1)
    qwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
    qwks.append(qwk)
    print qwk
    
    combinedModels.append(combinedModel)
    num += 1
    
meanQwk = quadratic_weighted_kappa.mean_quadratic_weighted_kappa(qwks)
print "Overall Test Qwk: %s" % meanQwk

Input shape: 1078
Adding Layer 0: 250
Adding 0.5 dropout
Adding Layer 1: 75
Adding 0.5 dropout
Adding Layer 2: 25
Adding 0.25 dropout
Epoch 1/50
38003/38003 [==============================] - 9s - loss: 1.7297     
Epoch 0 Kappa: 0.586490 | Best Kappa: 0.000000 

Epoch 2/50
38003/38003 [==============================] - 9s - loss: 1.5215     
Epoch 1 Kappa: 0.605599 | Best Kappa: 0.586490 

Epoch 3/50
38003/38003 [==============================] - 10s - loss: 1.4625    
Epoch 2 Kappa: 0.583192 | Best Kappa: 0.605599 

Epoch 4/50
38003/38003 [==============================] - 10s - loss: 1.4155    
Epoch 3 Kappa: 0.616938 | Best Kappa: 0.605599 

Epoch 5/50
38003/38003 [==============================] - 10s - loss: 1.3833    
Epoch 4 Kappa: 0.616216 | Best Kappa: 0.616938 

Epoch 6/50
25408/38003 [===================>..........] - ETA: 3s - loss: 1.3447

In [27]:
testPredictions = np.zeros(len(test))
# testModels = ['XGBoost']
for i in range(1, folds + 1):
    model = combinedModels[i-1]
    print model.cutPoints
#     testPredictions += model.predict(test, 'combinedTestPredictions%s.csv' % str(i), i)

    
# testPredictions /= folds
# predDf = pd.DataFrame()
# predDf['Id'] = dfTest['Id']
# predDf['Response'] = np.round(testPredictions).astype(int)
# print predDf['Response'].values
# predDf.to_csv(path_or_buf='NnWithXgBoost.csv', columns=['Id', 'Response'], index=False, header=['Id', 'Response'])

[ 0.70278949  3.57952058  4.55016914  5.19638604  5.89882557  6.58744627
  7.08271467]
[ 0.70278949  3.57952058  4.55016914  5.19638604  5.89882557  6.58744627
  7.08271467]
[ 0.70278949  3.57952058  4.55016914  5.19638604  5.89882557  6.58744627
  7.08271467]
[ 0.70278949  3.57952058  4.55016914  5.19638604  5.89882557  6.58744627
  7.08271467]
[ 0.70278949  3.57952058  4.55016914  5.19638604  5.89882557  6.58744627
  7.08271467]


In [7]:
kf = KFold(len(train), 5)
num = 1
qwks = list()

name = 'Keras'
model = nnModel = NN(inputShape = train.shape[1], cutPointArray=cpa, layers = [200, 75, 25], dropout = [0.5, 0.5, 0.5], activation='relu', patience=5, loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 50)

# model = XGBoostModel(700, 7, 0.025, 0.50, "reg:linear", 25)

for train_index, test_index in kf:
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)
    testFile = 'combinedTestPredictions%s.csv' % str(num)
    
    trainDF = pd.read_csv(trainFile) if os.path.isfile(trainFile) else pd.DataFrame()  
    validateDF = pd.read_csv(validateFile) if os.path.isfile(validateFile) else pd.DataFrame()  
    testDF = pd.read_csv(testFile) if os.path.isfile(testFile) else pd.DataFrame()  

    xTrain = train.iloc[train_index].values
    yTrain = labels.iloc[train_index]      
    xValidate = train.iloc[test_index].values
    yValidate = labels.iloc[test_index]
    
    model.fit(xTrain, yTrain, num-1)
    testPredictions = model.predict(test.values)
    trainPredictions = model.predict(xTrain)
    validatePredictions = model.predict(xValidate)
    
    trainDF[name] = trainPredictions
    validateDF[name] = validatePredictions
    testDF[name] = testPredictions
        
    trainDF.to_csv(path_or_buf=trainFile, index=False)
    validateDF.to_csv(path_or_buf=validateFile, index=False)
    testDF.to_csv(path_or_buf=testFile, index=False)
            
    num += 1

Input shape: 1078
Adding Layer 0: 200
Adding 0.5 dropout
Adding Layer 1: 75
Adding 0.5 dropout
Adding Layer 2: 25
Adding 0.5 dropout
Epoch 1/50
38003/38003 [==============================] - 9s - loss: 1.9125     
Epoch 0 Kappa: 0.564208 | Best Kappa: 0.000000 

Epoch 2/50
38003/38003 [==============================] - 9s - loss: 1.6355     
Epoch 1 Kappa: 0.592169 | Best Kappa: 0.564208 

Epoch 3/50
38003/38003 [==============================] - 10s - loss: 1.5241    
Epoch 2 Kappa: 0.567515 | Best Kappa: 0.592169 

Epoch 4/50
38003/38003 [==============================] - 11s - loss: 1.4376    
Epoch 3 Kappa: 0.595805 | Best Kappa: 0.592169 

Epoch 5/50
38003/38003 [==============================] - 11s - loss: 1.3830    
Epoch 4 Kappa: 0.581372 | Best Kappa: 0.595805 

Epoch 6/50
38003/38003 [==============================] - 11s - loss: 1.3434    
Epoch 5 Kappa: 0.568966 | Best Kappa: 0.595805 

Epoch 7/50
38003/38003 [==============================] - 10s - loss: 1.3163    
Epoch 

In [ ]:
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

keywords = ['Medical_Keyword_' + str(i) for i in range(1, 49)]
# print dfTrain[keywords].sum(axis=1)

print dfTrain[]['Response'].mean()
print dfTrain['Response'].mean()
#     Medical_History_10
# # uniqueValues = dfTest['InsuredInfo_7'].unique()
# # for i in range(len(uniqueValues)):
# #     arr = dfTrain['InsuredInfo_7'].apply(lambda x: x == uniqueValues[i])
    

# for column in ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6', 'Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5']:
   
#     if dfTrain[column].isnull().sum():
#         print column
#         print pearsonr(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
#         print dfTrain[column].median()
#         print len(dfTrain[dfTrain.Response == 8])
#         print len(dfTrain)
# #         print dfTrain[column]
# #         plt.plot(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
#         break
# # plt.show()
# # print dfTest['InsuredInfo_4'].unique()
# # print dfTrain['InsuredInfo_6'].apply(lambda x: x == 1)
# print dfTest['Medical_History_32'].isnull().sum()
            
# print dfTrain['Medical_History_32'].median()
# print dfTrain[dfTrain['Family_Hist_2'].notnull()]['Family_Hist_2'].max()
# # print dfTrain['Medical_History_32'].null().sum()
# # print dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'].median()
# # print len(pd.concat([dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32']]))
# # print pearsonr(dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Response'])

In [ ]:
neighborModel = GetBestModel(lambda: NearestNeighbors(), features)

In [ ]:
nearestNeighbors = neighborModel.kneighbors(dfTrain[features].values)

In [ ]:
predictions = np.zeros(len(dfTrain))
for i in range(len(dfTrain)):
#     predictions[i] = dfTrain.iloc[nearestNeighbors[1][i]].Response.mean()
    responses = dfTrain.iloc[nearestNeighbors[1][i]].Response
    weights = nearestNeighbors[0][i][4] - nearestNeighbors[0][i]
    predictions[i] = np.sum(responses * weights) / np.sum(weights)

In [ ]:
predictions = np.rint(predictions)
print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, dfTrain.Response)
print np.min(predictions)

In [ ]:
import scipy.stats as stats
lowerBound = np.mean(dfTrain['BMI']) - np.std(dfTrain['BMI'])
upperBound = np.mean(dfTrain['BMI']) + np.std(dfTrain['BMI'])
mean = np.mean(dfTrain['BMI'])
stats.normaltest(dfTrain['BMI'])

bmiScore = dfTrain['BMI'].apply(lambda x: max(0, lowerBound - x) if x < mean else max(0, x - upperBound))

model = LinearRegression(normalize=True)
features = ['Ins_Age', 'Wt', 'BMI']
X = np.ndarray((59381, 9))
X[:,0] = dfTrain['Ins_Age'].values
X[:,1] = dfTrain['BMI'].values
X[:,2] = dfTrain['Wt'].values
X[:,3] = dfTrain['Ins_Age'] * dfTrain['Wt']
X[:,4] = dfTrain['Ins_Age'] * dfTrain['BMI']
X[:,5] = dfTrain['Ins_Age'] * dfTrain['Ins_Age']
X[:,6] = dfTrain['Wt'] * dfTrain['Wt']
X[:,7] = dfTrain['Wt'] * dfTrain['BMI']
X[:,8] = dfTrain['Wt'] * dfTrain['Ins_Age']

model.fit(X, dfTrain['Response'])

In [ ]:
class CutPointOptimizer:
    
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual

    def qwk(self, cutPoints):
        transformedPredictions = np.searchsorted(cutPoints, self.predicted) + 1            
        return -1 * quadratic_weighted_kappa.quadratic_weighted_kappa(transformedPredictions, self.actual)

initialCutPoints = np.array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])

In [ ]:

# Product_Info_2
# Product_Info_3
# Employment_Info_2
# InsuredInfo_3
# Medical_History_2
# Medical_History_10

var = 'Product_Info_3'
for val in dfTrain[var].unique():
    hits = dfTrain[dfTrain[var] == val]
    print '%s, %s, %s' % (val, len(hits), np.mean(hits.Response))

In [ ]:
import theano
import theano.tensor as T
import quadratic_weighted_kappa
y1, y2 = T.vectors('y1', 'y2')
# s = 1 / (1 + T.exp(-x))
s = quadratic_weighted_kappa.quadratic_weighted_kappa(y1, y2)
# logistic = theano.function([y1, y2], s)

In [ ]:
#2 layer 200/75 cut points
cpa = np.ndarray((5, 7))
cpa[0,:] = [1.94094589,  2.77827061, 4.47922564,  5.59375133,  5.98938611,  6.86875996, 7.45790894]
cpa[1,:] = [1.93043217,  3.06875824,  4.8242938,   5.40717815,  6.17241788,  6.97729218, 7.46620894]
cpa[2,:] = [1.23315182,  3.39688586,  4.6380404,   5.37977014,  6.16224374,  6.73081194, 7.48322552]
cpa[3,:] = [2.33061803,  3.5055729,   4.52173829,  5.24184554,  6.1711789,   6.96746293, 7.7211743]
cpa[4,:] = [2.26716625,  3.59018596,  4.3780072,   5.21537676,  5.95488354,  6.72023874, 7.5133742]